In [1]:
import pandas as pd
import numpy as np
import re
import string

import spacy
# Load the language model
nlp = spacy.load('en_core_web_md')
from spacy.tokenizer import Tokenizer


# I. Data Cleaning

### Project Contents

1. [Data Cleaning](../code/1_data_cleaning.ipynb)
2. [Data Visualization](../code/2_eda.ipynb)[This Notebook]
3. [Classifcation Model Comparison](../code/3_modeling.ipynb)

---

### Notebook Purpose

This notebook is intended to process the "dirty work" of data science. We will parse and clean the text information from the scraped r/WallStreetBets and r/Investing subreddits--removing, modifying, and imputing features in a way that will provide for the best classifcation model. 

---

### Note on Data Collection

While our original intent was to scrape data from Reddit, recent changes and instability with the Pushshift API made this difficult. Given the time constraints for this deliverable, we chose to use provided data and thus there is no web scraping function or data provided in this notebook. 

---

### Raw Text to Useful Text

With the raw text retrieved from reddit, we need to further process this data before feeding it into our eventual classification model. The raw text has many attributes that are undesirable from a machine learning perspective, and in order to process this text into useful features, we will need to perform a few operations: 

1. Lower case case all words. 

2. Remove punctuation/line breaks etc. 

3. Impute or remove missing/deleted posts. 

4. Remove self-referential terms. 

5. Handle hyperlinks/emoji's/other non-traditional text items. 

6. Remove duplicated posts. 

7. Stem or lemmatize the text. 

Read in r/WallStreetBets and r/Investing dataframes

In [2]:
inv_df = pd.read_csv('../data/investing.csv')

wsb_df = pd.read_csv('../data/wallstreetbets.csv')

### Balance of Classes

In [3]:
inv_df.shape, wsb_df.shape

((10033, 4), (10052, 4))

We begin by evaluating the balance of classes between the two datasets. If there is an imbalance in the classes, we have one dataset that is over represented and another that is underrepresented. This allows the possibility of bias towards the majority class to creep into our model, and we run the risk of developing a model that will not accurately predict the minority class.

We see above that our datasets are quite balanced, which is a good thing. As we further clean the dataset we will continuously check this balance of classes and see what modifications if any are required. 

### Lowercase

In [4]:
# lower case all text

inv_df['title'] = inv_df['title'].str.lower()
wsb_df['title'] = wsb_df['title'].str.lower()

inv_df['selftext'] = inv_df['selftext'].str.lower()
wsb_df['selftext'] = wsb_df['selftext'].str.lower()

Since our models are case-sensitive, we'll lowercase all words in both the titles and selftext fields within both datasets so that we don't determine that words are different based on case rather than by their intrinsic nature. 

### Remove Punctuation

Regex found here [SOURCE](https://stackoverflow.com/questions/44524165/remove-punctuation-in-python-but-keep-emoticons/44524574#44524574)

In [5]:
# remove all punctuation 

inv_df['title'] = inv_df['title'].str.replace(r'(?<=\w)[^\s\w](?![^\s\w])','', regex=True)
inv_df['selftext'] = inv_df['selftext'].str.replace(r'(?<=\w)[^\s\w](?![^\s\w])','', regex=True)

wsb_df['title'] = wsb_df['title'].str.replace(r'(?<=\w)[^\s\w](?![^\s\w])','', regex=True)
wsb_df['selftext'] = wsb_df['selftext'].str.replace(r'(?<=\w)[^\s\w](?![^\s\w])','', regex=True)


We'll use a regular expression to strip all punctuation from the posts, but leave emoji's in. Emoji's play an integral role in the recent meme culture that has gripped millenial investors and their impact will be explored at depth below. 

The punctuation removal is accomplished via regular expression. 

### Remove \n and \r

In [6]:
# remove \n and \r from text

wsb_df['title'] = wsb_df['title'].str.replace(r'\n', ' ', regex=True)
wsb_df['selftext'] = wsb_df['selftext'].str.replace(r'\n', ' ', regex=True)

# same for investing
inv_df['title'] = inv_df['title'].str.replace(r'\n', ' ', regex=True)
inv_df['selftext'] = inv_df['selftext'].str.replace(r'\n', ' ', regex=True)

In [7]:

wsb_df['title'] = wsb_df['title'].str.replace(r'\r', ' ', regex=True)
wsb_df['selftext'] = wsb_df['selftext'].str.replace(r'\r', ' ', regex=True)

# same for investing
inv_df['title'] = inv_df['title'].str.replace(r'\r', ' ', regex=True)
inv_df['selftext'] = inv_df['selftext'].str.replace(r'\r', ' ', regex=True)

We remove all new line characters and carriage returns. 

### Missing/Deleted Posts

In [8]:
print(inv_df.isnull().sum())
print('')
print(wsb_df.isnull().sum())

title          0
selftext     314
author         0
subreddit      0
dtype: int64

title          0
selftext     170
author         0
subreddit      0
dtype: int64


In [9]:
# indices of wsb posts with null values
wsb_null_idx = wsb_df[wsb_df.isnull().any(axis=1)].index
display(wsb_df.iloc[wsb_null_idx])

# indices of investing posts with null values
inv_null_idx = inv_df[inv_df.isnull().any(axis=1)].index
display(inv_df.iloc[inv_null_idx])

,title,selftext,author,subreddit
58,5 comments and ill show my fellow degenerates ...,NaN,[deleted],wallstreetbets
396,testtging!!!!!!!,NaN,[deleted],wallstreetbets
464,where are the people who bought $fsly puts,NaN,[deleted],wallstreetbets
857,whats the deal with tesla,NaN,[deleted],wallstreetbets
1016,how to thrive through end of 2020,NaN,[deleted],wallstreetbets
...,...,...,...,...
9471,is tesla the new bitcoin bubble,NaN,[deleted],wallstreetbets
9474,space x is gonna start paying tesla for exclus...,NaN,[deleted],wallstreetbets
9578,the degenerate wager,NaN,[deleted],wallstreetbets
9580,what is a great resource for someone who is co...,NaN,sirtones1411,wallstreetbets


,title,selftext,author,subreddit
373,apple during the event hold or sell,NaN,[deleted],investing
810,wtf happened here,NaN,[deleted],investing
812,stocks turn negative after trump says white ho...,NaN,[deleted],investing
858,have any stretches helped you with pain,NaN,[deleted],investing
1137,im up 35 this year but im going back to index ...,NaN,[deleted],investing
...,...,...,...,...
10009,citadel securities is cashing in on the retail...,NaN,[deleted],investing
10010,zerofee trading helps citadel securities cash ...,NaN,[deleted],investing
10013,us home sales tumble to 912year low price grow...,NaN,[deleted],investing
10024,how do you get data for your analysis,NaN,[deleted],investing


r/Investing has 314 posts that are missing selftext and r/WallStreetBets has 170 posts that are missing selftext. The dataframes above demonstrate that is due to the posts being deleted. Since, we have the luxury of working with very popular subreddits and therefore large datasets, we can delete the missing posts and not substantially alter our dataset.

Drop Missing Values

In [10]:
wsb_df.dropna(inplace=True)
inv_df.dropna(inplace=True)

wsb_df.shape, inv_df.shape

((9882, 4), (9719, 4))

### Duplicated Posts

In [11]:
# wsb duplicates
print(wsb_df.duplicated().sum())
wsb_df[wsb_df.duplicated()]

# drop duplicates
wsb_df.drop_duplicates(inplace=True)

23


In [12]:
# investing duplicates
print(inv_df.duplicated().sum())
inv_df[inv_df.duplicated()]


# drop duplicates
inv_df.drop_duplicates(inplace=True)

159


We check for any double posts by using the pandas `drop_duplicates` function and remove the double posts. 

### Redact Posts that have "WSB" in the subject or body


In [13]:
# find posts within r/wallstreetbets that have have "wsb" in the title or body 

print('WSB in Title :', wsb_df['title'].str.contains('wsb', case=False).sum())

print('WSB in Body :', wsb_df['selftext'].str.contains('wsb', case=False).sum())

WSB in Title : 246
WSB in Body : 670


In [14]:
# find posts within r/investing that have have "wsb" in the title or body

print('WSB in Title :', inv_df['title'].str.contains('wsb', case=False).sum())

print('WSB in Body :', inv_df['selftext'].str.contains('wsb', case=False).sum())


WSB in Title : 9
WSB in Body : 68



It is important to remove any self-referential terms in the bodies of our datapoints, otherwise the algorithm will not have real predictive power. 

There are 246 posts that have 'wsb' in the title and 670 posts that have 'wsb' in the subject within r/WallStreetBets. Additionally, there are 8 posts that have 'wsb' within the title in r/Investing and 52 that have 'wsb' in the in the subject. Rather than remove the posts entirely, we'll just replace 'wsb' with whitespace. 

In [15]:
# find posts that have "wsb" in the body and replace it with whitespace
wsb_df['selftext'] = wsb_df['selftext'].str.replace('wsb', ' ', case=False)

# find posts that have "wsb" in the title and replace it with whitespace
wsb_df['title'] = wsb_df['title'].str.replace('wsb', ' ', case=False)

# check for redaction
print('WSB in Title :', wsb_df['title'].str.contains('wsb', case=False).sum())
print('WSB in Body :', wsb_df['selftext'].str.contains('wsb', case=False).sum())

WSB in Title : 0
WSB in Body : 0


In [16]:
# find posts that have "wsb" in the body and replace it with whitespace
inv_df['selftext'] = inv_df['selftext'].str.replace('wsb', ' ', case=False)

# find posts that have "wsb" in the title and replace it with whitespace
inv_df['title'] = inv_df['title'].str.replace('wsb', ' ', case=False)

# check for redaction
print('WSB in Title :', inv_df['title'].str.contains('wsb', case=False).sum())
print('WSB in Body :', inv_df['selftext'].str.contains('wsb', case=False).sum())

WSB in Title : 0
WSB in Body : 0


### Presence of URL's

In [17]:
# posts with a link in the selftext
print('Number of r/inv posts with links in selftext: ', len(inv_df[inv_df['selftext'].str.contains('http', case=False)]))

# posts with a link in the title
print('Number of r/inv posts with links in title: ',len(inv_df[inv_df['title'].str.contains('http', case=False)]))

Number of r/inv posts with links in selftext:  2157
Number of r/inv posts with links in title:  6


In [18]:
# posts with a link in the selftext
print('Number of r/wsb posts with links in selftext: ', len(wsb_df[wsb_df['selftext'].str.contains('http', case=False)]))

# posts with a link in the title
print('Number of r/wsb posts with links in title: ',len(wsb_df[wsb_df['title'].str.contains('http', case=False)]))

Number of r/wsb posts with links in selftext:  2314
Number of r/wsb posts with links in title:  0


Both r/WallStreetBets and r/Investing have over 2,000 links contained in their respective selftexts, which accounts for roughly 20% of the respective datasets. This is as expected, as both subreddits center on investing and it stands to reason that the participants would share market news in their posts. Rather than remove these links, we will retain them and simply redact them with the string 'URL'.

This task is accomplished below in the SpACy portion of the notebook. 

### Emoji's

In [19]:
# presence of emojis in wsb df

wsb_emojis = ['🚀', '🙌', '🐻', '🐂', '💵', '🍗','💎']

def go_brr(df):
    emoji_dict = {}
    for emoji in wsb_emojis:
        emoji_dict[emoji] = len(df['title'].str.findall(emoji).sum()) + len(df['selftext'].str.findall(emoji).sum())
    return emoji_dict

print(go_brr(wsb_df))
print(go_brr(inv_df))

{'🚀': 649, '🙌': 8, '🐻': 385, '🐂': 19, '💵': 7, '🍗': 9, '💎': 24}
{'🚀': 26, '🙌': 0, '🐻': 0, '🐂': 0, '💵': 6, '🍗': 0, '💎': 0}


Emoji's are an integral part of WSB culture and frequent posters will routinely express their thoughts and emotions in a combination of the emoji's. 

For example 💎🙌 represents *Diamond Hands* or the idea that the poster will hold on to their chosen security through periods of great volatility or  drawdown. 🍗 is a meme within a meme, WallStreetBettors often refer to cash as "tendies", so the chicken leg is a derivation that flows from Legal Tender -> Tender -> Tendies -> Chicken Tender -> Chicken Tendies -> 🍗. The memeification of investment language and sentiment is so integral to WSB culture that we will have to keep the emoji's in the dataset. 

### Remove any extra whitespace

In [20]:
# wsb
wsb_df['selftext'] = wsb_df['selftext'].str.strip() 
wsb_df['title'] = wsb_df['title'].str.strip() 

# investing
inv_df['selftext'] = inv_df['selftext'].str.strip() 
inv_df['title'] = inv_df['title'].str.strip() 

We strip out any additional whitespace that might trip up our classifiers. 

### Use SpAcy to Lemmatize text, but retain the Emojis and convert hyperlinks to 'URL'

In [21]:
# nlp.add_pipe("emoji", first=True)

# Create a custom tokenizer that handles emojis and URLs
# This is a variation on code I found here (https://stackoverflow.com/questions/51012476/spacy-custom-tokenizer-to-include-only-hyphen-words-as-tokens-using-infix-regex)

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=nlp.tokenizer.infix_finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)

# Set the custom tokenizer as the tokenizer for the nlp object
nlp.tokenizer = custom_tokenizer(nlp)

def lemmatize_text_with_emojis_and_urls(text):
    doc = nlp(text)
    lemmatized_tokens = []
    for token in doc:
        if token.like_url:
            lemmatized_tokens.append("URL")
        elif token.is_punct:
            lemmatized_tokens.append(token.text)
        else:
            lemmatized_tokens.append(token.lemma_.lower())

    return ' '.join(lemmatized_tokens)

# Apply the lemmatize_text_with_emojis_and_urls function to the 'text' column of the DataFrame
for df in [wsb_df, inv_df]:
    df['selftext_lemmatized'] = df['selftext'].apply(lemmatize_text_with_emojis_and_urls)
    df['title_lemmatized'] = df['title'].apply(lemmatize_text_with_emojis_and_urls)


### Word Processing

We will use the spaCy library to further process our text. 

**Tokenization**

Tokenization is the process of reducing a body of text into smaller elements, for example turning a sentence into a list of individual words. With these tokenized words, we further process them. 

**Stemming vs. Lemmatization**

We consider the usage of two word processing methods, Stemming and Lemmatization. Stemming removes or "stems" characters from a word, without referencing the context of the word. Lemmatization is the act of reducing the word to its base form, which we call a Lemma. An example of the lemmatization vs. stemming process would be the processing the word *Caring*. Lemmatization will return *Care* whereas stemming will return *Car*, which could lead to erroneous interpretations. [SOURCE](https://stackoverflow.com/questions/1787110/what-is-the-difference-between-lemmatization-vs-stemming).

Lemmatization is much more computationally expensive than Stemming, however our dataset is not so large as to rule out its practicality so we will use Lemmatization due to its superior accuracy. 


The custom tokenization and lemmatization process above will not remove emoticons, for the reasons discussed above, and will convert all hyperlinks into the string value `URL`.  

In [22]:
# remove all ^ symbols

for df in [wsb_df, inv_df]:
    df['selftext_lemmatized'] = df['selftext_lemmatized'].str.replace('^', ' ', case=False, regex=False)
    df['title_lemmatized'] = df['title_lemmatized'].str.replace('^', ' ', case=False, regex=False)

The cleaning process has been effective but has still left some `^` marks where the hyperlinks once were. We will replace them with whitespace. 

---

### Write cleaned data to .CSV

In [23]:
# r/wallstreetbets
wsb_df.to_csv('../data/wsb_df_clean.csv', index=False)

# r/investing
inv_df.to_csv('../data/inv_df_clean.csv', index=False)

In order to save time in the EDA and modelling evolutions, we will save our cleaned data to `.csv` files in our data directory. 